In [3]:
import h3
import pandas as pd
import matplotlib
import folium

from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster

In [2]:
import branca.colormap as cm
from geojson.feature import *
import json

In [4]:
import os
import sys

In [51]:
sys.path.append(os.path.abspath('data/yellow_tripdata_2016-04.csv'))

In [52]:
columns = ['tpep_pickup_datetime'
           , 'tpep_dropoff_datetime'
           , 'trip_distance'
           , 'pickup_longitude'
           , 'pickup_latitude'
           , 'dropoff_longitude'
           , 'dropoff_latitude'
           , 'fare_amount']

In [53]:
del df
del fltrd_1

NameError: name 'fltrd_1' is not defined

In [54]:
df = pd.read_csv(os.path.abspath('../data/yellow_tripdata_2016-04.csv')
                 , usecols=columns
                 , parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
                #, nrows=500
                )

In [55]:
def timestamp_converter(timestamps, interval=10):
    '''
    :param: timestamps specifies the (pandas) Series of timestamps to convert
    :param: interval=10 means that timestamp is ceiled to 10 min interval (e.g. 00:03:59 -> 00:10:00)
    
    '''
    ceiled = timestamps.dt.ceil("{}min".format(interval))
    return ceiled.apply(lambda x : int('%d%02d' % (x.hour, x.minute)))
    #return ceiled #.apply(lambda x : int('%d%02d' % (x.hour, x.minute))/10)

In [56]:
df['pickup_time'] = timestamp_converter(df['tpep_pickup_datetime'], interval=10)
df['dropoff_time'] = timestamp_converter(df['tpep_dropoff_datetime'], interval=10)

In [57]:
rng = pd.date_range('00:00:00', '23:59:59', freq="10min")
s = pd.DataFrame({'pickup_time':rng})
s['pu_city_time'] = s.index
s['pickup_time'] = s['pickup_time'].apply(lambda x : int('%d%02d' % (x.hour, x.minute)))

t = pd.DataFrame({'dropoff_time':rng})       
t['do_city_time'] = t.index
t['dropoff_time'] = t['dropoff_time'].apply(lambda x : int('%d%02d' % (x.hour, x.minute)))
        
df = df.merge(s, on='pickup_time', how='left')
df = df.merge(t, on='dropoff_time', how='left')
#df

In [58]:
lat_min = 40.731461
lat_max = 40.782619
lon_min = -74.008574
lon_max = -73.950370

In [60]:
#del fltrd_1 

fltrd_1 = df.loc[
                (df.pickup_longitude >= lon_min)  & \
                (df.pickup_longitude <= lon_max)  & \
                (df.pickup_latitude >= lat_min)   & \
                (df.pickup_latitude <= lat_max)   & \
                (df.dropoff_longitude >= lon_min) & \
                (df.dropoff_longitude <= lon_max) & \
                (df.dropoff_latitude >= lat_min)  & \
                (df.dropoff_latitude <= lat_max)
                ].copy()
len(fltrd_1)

5466220

In [329]:
fltrd_1.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,pickup_time,dropoff_time,pu_city_time,do_city_time,pick_hex_id,drop_hex_id
0,2016-04-01 00:00:00,2016-04-01 00:01:59,0.50,-73.976883,40.758495,-73.977669,40.753902,3.5,0,10,0,1,892a100d60fffff,892a100d677ffff
1,2016-04-01 00:00:00,2016-04-01 00:12:07,2.20,-73.985207,40.757294,-73.989288,40.732658,10.0,0,20,0,2,892a100d67bffff,892a100d22bffff
2,2016-04-01 00:00:00,2016-04-01 00:10:41,0.96,-73.979202,40.758869,-73.990677,40.751320,8.5,0,20,0,2,892a100d673ffff,892a100d2dbffff
3,2016-04-01 00:00:00,2016-04-01 00:10:30,1.54,-73.984856,40.767723,-73.990829,40.751186,8.5,0,20,0,2,892a1008b27ffff,892a100d2c3ffff
7,2016-04-01 00:00:01,2016-04-01 00:03:46,0.60,-73.988899,40.745426,-73.991821,40.738445,4.5,10,10,1,1,892a100d21bffff,892a100d273ffff


In [61]:
import h3.api.numpy_int as h3_np
fltrd_1["pick_hex_id"] = fltrd_1.apply(lambda row: h3_np.geo_to_h3(row['pickup_latitude'], row['pickup_longitude'], 9), axis = 1)
fltrd_1["drop_hex_id"] = fltrd_1.apply(lambda row: h3_np.geo_to_h3(row['dropoff_latitude'], row['dropoff_longitude'], 9), axis = 1)

In [63]:
fltrd_1["h3_distance"] = fltrd_1.apply(lambda row: h3_np.h3_distance(row['pick_hex_id'], row['drop_hex_id']), axis=1)

In [80]:
print('columns: {},\n length: {} rows, \n nr. cells {}'.format(fltrd_1.columns.values, len(fltrd_1), len(fltrd_1.pick_hex_id.unique())))

columns: ['tpep_pickup_datetime' 'tpep_dropoff_datetime' 'trip_distance'
 'pickup_longitude' 'pickup_latitude' 'dropoff_longitude'
 'dropoff_latitude' 'fare_amount' 'duration' 'pick_hex_id' 'drop_hex_id'
 'pick_geometry' 'drop_geometry'],
 length: 4390 rows, 
 nr. cells 388


In [64]:
fltrd_1.to_csv(os.path.abspath('../data/prep_data_2016-04.csv'))

In [65]:
del df
del fltrd_1

In [342]:
fltrd_1 = pd.read_csv(os.path.abspath('../data/prep_data_2016-04.csv')
                      , usecols=['pickup_time', 'pick_hex_id']
                      , nrows=500
)
                    
fltrd_1.head()

,pickup_time,pick_hex_id
0,0,892a100d60fffff
1,0,892a100d67bffff
2,0,892a100d673ffff
3,0,892a1008b27ffff
4,10,892a100d21bffff


In [343]:
fltrd_1["pick_geometry"] =  fltrd_1.pick_hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(h=x,geo_json=True)]
                                                            }
                                                        )

In [74]:
fltrd_1["drop_geometry"] =  fltrd_1.drop_hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(h=x,geo_json=True)]
                                                            }
                                                        )

In [344]:
def counts_by_hexagon(df, columns=None, resolution=12):
    """
    Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
    Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons
    
    Ex counts_by_hexagon(data, 9)
    
    resotlution (default 9)
    """
#    if columns:
#        df = df[columns]     
#    else:
#        df = df[["lat","lng"]]
#    
#    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row[df.columns[0]], row[df.columns[1]], resolution), axis = 1)
    
    df_aggreg = df.groupby(by = "pick_hex_id").size().reset_index()
    df_aggreg.columns = ["pick_hex_id", "value"]
    
    df_aggreg["geometry"] =  df_aggreg.pick_hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(h=x,geo_json=True)]
                                                            }
                                                        )
    
    return df_aggreg

In [345]:
def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
                   kind = "linear"):
    
    """
    Creates choropleth maps given the aggregated data.
    """    
    #colormap
    min_value = df_aggreg["value"].min()
    max_value = df_aggreg["value"].max()
    m = round ((min_value + max_value ) / 2 , 0)

    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'pick_hex_id'])
    
    if initial_map is None:
        initial_map = Map(location= [40.718728, -73.943802], zoom_start=12, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
        

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)
    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)
    
    
    
    return initial_map

In [347]:
def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    """
    Produce the GeoJSON for a dataframe that has a geometry column in geojson 
    format already, along with the columns hex_id and value
    
    Ex counts_by_hexagon(data)
    """    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["pick_hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
        #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result

In [348]:
df_aggreg = counts_by_hexagon(df=fltrd_1, columns=['pickup_latitude', 'pickup_longitude'], resolution=1)
df_aggreg.sort_values(by="value", ascending=False, inplace=True)
hexmap = choropleth_map(df_aggreg=df_aggreg, with_legend=True)
hexmap.save('hexmap.html')
hexmap

In [61]:
len(df)

11934338

In [60]:
fltrd_1.pickup_longitude.min()

-74.03138732910156

In [ ]:
fltrd_2 = df[(df.tpep_pickup_datetime != df.tpep_dropoff_datetime)]

In [52]:
fltrd_df = df.query(query)
fltrd_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount


In [35]:
print(df['tpep_pickup_datetime'][4])
print(df['tpep_dropoff_datetime'][4])

2016-04-01 00:00:00
2016-04-01 00:00:00


In [16]:
df.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount'],
      dtype='object')

In [25]:
df.dtypes

tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
trip_distance                   float64
pickup_longitude                float64
pickup_latitude                 float64
dropoff_longitude               float64
dropoff_latitude                float64
fare_amount                     float64
dtype: object

In [20]:
df['pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df.drop(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
df.dtypes

tpep_pickup_datetime             object
tpep_dropoff_datetime            object
trip_distance                   float64
pickup_longitude                float64
pickup_latitude                 float64
dropoff_longitude               float64
dropoff_latitude                float64
fare_amount                     float64
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
dtype: object

In [21]:
df.columns

Index(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'pickup_datetime',
       'dropoff_datetime'],
      dtype='object')

In [17]:
del df, fltrd_2

NameError: name 'df' is not defined

In [332]:
fltrd_1.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,pickup_time,dropoff_time,pu_city_time,do_city_time,pick_hex_id,drop_hex_id,h3_distance
0,2016-04-01 00:00:00,2016-04-01 00:01:59,0.50,-73.976883,40.758495,-73.977669,40.753902,3.5,0,10,0,1,892a100d60fffff,892a100d677ffff,1
1,2016-04-01 00:00:00,2016-04-01 00:12:07,2.20,-73.985207,40.757294,-73.989288,40.732658,10.0,0,20,0,2,892a100d67bffff,892a100d22bffff,9
2,2016-04-01 00:00:00,2016-04-01 00:10:41,0.96,-73.979202,40.758869,-73.990677,40.751320,8.5,0,20,0,2,892a100d673ffff,892a100d2dbffff,4
3,2016-04-01 00:00:00,2016-04-01 00:10:30,1.54,-73.984856,40.767723,-73.990829,40.751186,8.5,0,20,0,2,892a1008b27ffff,892a100d2c3ffff,7
7,2016-04-01 00:00:01,2016-04-01 00:03:46,0.60,-73.988899,40.745426,-73.991821,40.738445,4.5,10,10,1,1,892a100d21bffff,892a100d273ffff,2


In [66]:
columns = [ 'pick_hex_id'
          , 'drop_hex_id'
          , 'pu_city_time'
          , 'do_city_time'
          , 'h3_distance'
          , 'fare_amount']

In [67]:
df = pd.read_csv(os.path.abspath('../data/prep_data_2016-04.csv')
                 , usecols=columns
                #, nrows=500
                )[columns]

In [68]:
fltrd_2 = df.loc[
                (df.do_city_time > df.pu_city_time) & \
                (df.pick_hex_id != df.drop_hex_id) & \
                (df.fare_amount >= 1.0)
                #(df.dropoff_latitude <= lat_max)  & \
                ].copy()
fltrd_2.head()

,pick_hex_id,drop_hex_id,pu_city_time,do_city_time,h3_distance,fare_amount
0,617733123866623999,617733123873439743,0,1,1,3.5
1,617733123873701887,617733123801350143,0,2,9,10.0
2,617733123873177599,617733123812884479,0,2,4,8.5
3,617733122609905663,617733123811311615,0,2,7,8.5
7,617733123868459007,617733122576351231,1,2,8,10.0


In [69]:
len(fltrd_2)

4186355

In [70]:
fltrd_2.dtypes

pick_hex_id       int64
drop_hex_id       int64
pu_city_time      int64
do_city_time      int64
h3_distance       int64
fare_amount     float64
dtype: object

In [71]:
fltrd_2.describe()

,pick_hex_id,drop_hex_id,pu_city_time,do_city_time,h3_distance,fare_amount
count,4.186355e+06,4.186355e+06,4.186355e+06,4.186355e+06,4.186355e+06,4.186355e+06
mean,6.177331e+17,6.177331e+17,8.623864e+01,8.759761e+01,6.113356e+00,9.638284e+00
std,9.349250e+09,9.596211e+09,3.447567e+01,3.448817e+01,3.206620e+00,3.915534e+00
min,6.177331e+17,6.177331e+17,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,6.177331e+17,6.177331e+17,6.200000e+01,6.300000e+01,4.000000e+00,7.000000e+00
50%,6.177331e+17,6.177331e+17,8.800000e+01,9.000000e+01,5.000000e+00,9.000000e+00
75%,6.177331e+17,6.177331e+17,1.150000e+02,1.160000e+02,8.000000e+00,1.150000e+01
max,6.177332e+17,6.177332e+17,1.420000e+02,1.430000e+02,2.300000e+01,1.411000e+03


In [72]:
fltrd_3 = fltrd_2[['pick_hex_id', 'pu_city_time']]
grpd = fltrd_3.groupby(['pick_hex_id', 'pu_city_time'])
grpd.size()

pick_hex_id         pu_city_time
617733122573991935  0                20
                    1                20
                    2                13
                    3                 5
                    4                 9
                                   ... 
617733151092637695  138             210
                    139             206
                    140             188
                    141             162
                    142             143
Length: 29560, dtype: int64

In [75]:
arr = fltrd_2.to_numpy()
arr

array([[6.17733124e+17, 6.17733124e+17, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 3.50000000e+00],
       [6.17733124e+17, 6.17733124e+17, 0.00000000e+00, 2.00000000e+00,
        9.00000000e+00, 1.00000000e+01],
       [6.17733124e+17, 6.17733124e+17, 0.00000000e+00, 2.00000000e+00,
        4.00000000e+00, 8.50000000e+00],
       [6.17733123e+17, 6.17733124e+17, 0.00000000e+00, 2.00000000e+00,
        7.00000000e+00, 8.50000000e+00],
       [6.17733124e+17, 6.17733123e+17, 1.00000000e+00, 2.00000000e+00,
        8.00000000e+00, 1.00000000e+01]])

In [77]:
import numpy as np
np.save(os.path.abspath('../data/prep_data.npy'), arr)

In [78]:
brr = np.load(os.path.abspath('../data/prep_data.npy'))

array([[6.17733124e+17, 6.17733124e+17, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 3.50000000e+00],
       [6.17733124e+17, 6.17733124e+17, 0.00000000e+00, 2.00000000e+00,
        9.00000000e+00, 1.00000000e+01],
       [6.17733124e+17, 6.17733124e+17, 0.00000000e+00, 2.00000000e+00,
        4.00000000e+00, 8.50000000e+00],
       ...,
       [6.17733151e+17, 6.17733124e+17, 1.15000000e+02, 1.16000000e+02,
        1.00000000e+00, 5.00000000e+00],
       [6.17733124e+17, 6.17733124e+17, 1.15000000e+02, 1.16000000e+02,
        4.00000000e+00, 7.50000000e+00],
       [6.17733151e+17, 6.17733123e+17, 1.15000000e+02, 1.17000000e+02,
        1.30000000e+01, 1.30000000e+01]])

In [82]:
len(brr)

4186355

In [129]:
for x in range(500):
    if brr[x,0] == 617733122573991935:
        print(brr[x,:])
#brr[0,0]

[6.17733123e+17 6.17733124e+17 8.80000000e+01 9.00000000e+01
 1.00000000e+01 1.20000000e+01]
[6.17733123e+17 6.17733123e+17 8.80000000e+01 8.90000000e+01
 3.00000000e+00 7.50000000e+00]
[6.17733123e+17 6.17733151e+17 8.80000000e+01 9.10000000e+01
 1.20000000e+01 1.60000000e+01]


In [ ]:
import random
nrr = random.sample(brr)

In [436]:
brr[1400674,:]

array([6.17733123e+17, 6.17733124e+17, 4.00000000e+01, 4.10000000e+01,
       8.00000000e+00, 9.00000000e+00])

In [433]:
int(6.17733124e+17)

617733124000000000